In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb

In [17]:
import os
os.chdir("D:\\elliptic\\Elliptic_Dataset")
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("txs_features.csv")
print(df_txs_features)

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("txs_classes.csv")
print(df_txs_classes)

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("txs_edgelist.csv")

print(df_txs_edgelist)


Transaction features: 

             txId  Time step  Local_feature_1  Local_feature_2  \
0            3321          1        -0.169615        -0.184668   
1           11108          1        -0.137586        -0.184668   
2           51816          1        -0.170103        -0.184668   
3           68869          1        -0.114267        -0.184668   
4           89273          1         5.202107        -0.210553   
...           ...        ...              ...              ...   
203764  158304003         49        -0.165622        -0.139563   
203765  158303998         49        -0.167040        -0.139563   
203766  158303966         49        -0.167040        -0.139563   
203767  161526077         49        -0.172212        -0.139573   
203768  194103537         49        -0.172212        -0.139573   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
0             -1.201369        -0.121970        -0.043875        -0.113002   
1             -1.201369   

In [18]:
print("\ntxs_features.csv for txId = 272145560\n")
print(df_txs_features[df_txs_features['txId']==272145560])

print("\ntxs_classes.csv for txId = 272145560\n")
print(df_txs_classes[df_txs_classes['txId']==272145560])

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId = 272145560

             txId  Time step  Local_feature_1  Local_feature_2  \
105573  272145560         24        -0.155493        -0.107012   

        Local_feature_3  Local_feature_4  Local_feature_5  Local_feature_6  \
105573        -1.201369         -0.12197        -0.043875        -0.113002   

        Local_feature_7  Local_feature_8  ...  in_BTC_min  in_BTC_max  \
105573        -0.061584        -0.145749  ...      2.7732      2.7732   

        in_BTC_mean  in_BTC_median  in_BTC_total  out_BTC_min  out_BTC_max  \
105573       2.7732         2.7732        2.7732     0.001917     2.770883   

        out_BTC_mean  out_BTC_median  out_BTC_total  
105573        1.3864          1.3864         2.7728  

[1 rows x 184 columns]

txs_classes.csv for txId = 272145560

             txId  class
105573  272145560      1

txs_edgelist.csv for txId = 272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560


In [19]:
df_txs_features = df_txs_features.merge(df_txs_classes, left_on = "txId" , right_on = "txId" , how = "left")
# print(txs.shape)

print(df_txs_features["class"].value_counts())

class
3    157205
2     42019
1      4545
Name: count, dtype: int64


In [20]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932,3


In [21]:
for column in df_txs_features.columns[168:]:
    if column != "class":
        feature = np.array(df_txs_features[column]).reshape(-1,1)
        scaler = MinMaxScaler()
        scaler.fit(feature)
        feature_scaled = scaler.transform(feature)
        df_txs_features[column] = feature_scaled.reshape(1,-1)[0]

In [22]:
# remove 'unknown' transactions
data = df_txs_features["txId"] #.loc[(df_txs_features['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features.loc[df_txs_features['txId'].isin(data)]
print(df_txs_features_selected["class"].value_counts())

class
3    156759
2     41500
1      4545
Name: count, dtype: int64


In [23]:
df_txs_features_selected

,txId,Time step,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,Local_feature_8,...,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total,class
0,3321,1,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160199,...,0.000047,0.000047,0.000047,0.000047,8.301504e-05,0.000032,0.000089,8.904096e-05,0.000047,3
1,11108,1,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.127429,...,0.000493,0.000493,0.000493,0.000493,2.915711e-04,0.000444,0.000936,9.357923e-04,0.000493,3
2,51816,1,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.160699,...,0.000040,0.000040,0.000040,0.000040,1.134016e-04,0.000020,0.000076,7.612341e-05,0.000040,3
3,68869,1,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,-0.161652,...,0.000702,0.000272,0.000088,0.000817,6.113009e-04,0.000714,0.001552,1.552291e-03,0.000817,2
4,89273,1,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,5.335864,...,0.074805,0.074805,0.074805,0.074805,6.466160e-11,0.003648,0.000022,1.451405e-07,0.074805,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.584665,...,0.010179,0.010179,0.010179,0.010179,8.223464e-04,0.010104,0.019336,1.933576e-02,0.010179,3
202800,194747925,49,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.573676,...,0.010029,0.010029,0.010029,0.010029,1.012352e-05,0.010098,0.019052,1.905181e-02,0.010029,3
202801,194748063,49,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.569524,...,0.009973,0.009973,0.009973,0.009973,4.604647e-04,0.009961,0.018945,1.894453e-02,0.009973,3
202802,194748070,49,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,0.563549,...,0.009891,0.009891,0.009891,0.009891,1.505606e-04,0.009935,0.018790,1.879015e-02,0.009891,3


In [24]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

X_data = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_training_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data)]
X_train = X_training_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35), 'txId']
X_testing_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test)]
X_test = X_testing_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test_class01 = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_testing_timesteps_class01 = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test_class01)]
X_test_class01 = X_testing_timesteps_class01.drop(columns=['txId', 'class', 'Time step'])

y_training_timesteps = X_training_timesteps[['class']]
y_training_timesteps = y_training_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_train = y_training_timesteps

y_testing_timesteps = X_testing_timesteps[['class']]
y_testing_timesteps = y_testing_timesteps['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test = y_testing_timesteps

y_testing_timesteps_class01 = X_testing_timesteps_class01[['class']]
y_testing_timesteps_class01 = y_testing_timesteps_class01['class'].apply(lambda x: x-1 ) # change illicit (class-2) to '0' for classification
y_test_class01 = y_testing_timesteps_class01

In [25]:
y_train.value_counts()

class
1    26237
0     3462
Name: count, dtype: int64

In [26]:

import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data
from sklearn.metrics import precision_recall_fscore_support, f1_score, confusion_matrix

# ---------------------------
# Config
# ---------------------------
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TRAIN_TS = list(range(1, 35))  # Time step < 35  => 1..34
TEST_TS  = list(range(35, 50)) # Time step >=35 => 35..49

# ---------------------------
# 1) Build snapshots (list of Data) for a list of timesteps
# ---------------------------
def build_snapshots(df_features, df_edges, timesteps, keep_unknown=False):
    """
    Returns:
      - snapshots: list of torch_geometric.data.Data (in ascending timestep order)
      - node_maps: list of dict mapping txId -> local_idx for each snapshot (same order)
      - ts_list: list of corresponding timesteps
    Notes:
      - We filter out nodes with class==3 unless keep_unknown=True
      - feature columns: all columns except ['txId','Time step','class']
    """
    snapshots = []
    node_maps = []
    ts_list = []

    # Precompute feature columns
    reserved_cols = ['txId', 'Time step', 'class']
    feature_cols = [c for c in df_features.columns if c not in reserved_cols]

    for t in sorted(timesteps):
        # Filter nodes at timestep t
        nodes_t = df_features[df_features['Time step'] == t].copy()
        if not keep_unknown:
            nodes_t = nodes_t[nodes_t['class'] != 3]

        if nodes_t.shape[0] == 0:
            # empty snapshot (rare) -> skip
            continue

        txids = nodes_t['txId'].values
        id2idx = {int(tx): i for i, tx in enumerate(txids)}

        # Build edges: only edges where both ends are in this snapshot
        if 'timestep' in df_edges.columns:
            ed_t = df_edges[df_edges['timestep'] == t].copy()
        else:
            # if no timestep column, select edges whose txIds are in this snapshot
            ed_t = df_edges.copy()
        # keep only edges inside this snapshot
        ed_t = ed_t[ed_t['txId1'].isin(id2idx) & ed_t['txId2'].isin(id2idx)]

        if len(ed_t) > 0:
            # build edge_index with directed edges (keep original direction)
            src = [id2idx[int(x)] for x in ed_t['txId1'].values]
            dst = [id2idx[int(x)] for x in ed_t['txId2'].values]
            edge_index = torch.tensor([src, dst], dtype=torch.long)
            # optionally make undirected by concatenation:
            # edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
        else:
            edge_index = torch.empty((2, 0), dtype=torch.long)

        # Features and labels
        X = torch.tensor(nodes_t[feature_cols].values, dtype=torch.float)
        # map labels: class 1->0, class 2->1  (as your earlier code did class-1)
        y = torch.tensor((nodes_t['class'].values - 1).astype(np.int64), dtype=torch.long)

        data = Data(x=X, edge_index=edge_index, y=y)
        snapshots.append(data)
        node_maps.append(id2idx)
        ts_list.append(t)

    return snapshots, node_maps, ts_list

# Build train & test snapshot lists
train_snapshots, train_maps, train_ts = build_snapshots(df_txs_features_selected, df_txs_edgelist, TRAIN_TS, keep_unknown=False)
test_snapshots,  test_maps,  test_ts  = build_snapshots(df_txs_features_selected, df_txs_edgelist, TEST_TS,  keep_unknown=False)

print(f"Built {len(train_snapshots)} train snapshots (timesteps {train_ts[:3]}...{train_ts[-3:]})")
print(f"Built {len(test_snapshots)} test snapshots (timesteps {test_ts[:3]}...{test_ts[-3:]})")

Built 34 train snapshots (timesteps [1, 2, 3]...[32, 33, 34])
Built 15 test snapshots (timesteps [35, 36, 37]...[47, 48, 49])


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class EvolveGCNH(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers=2):
        """
        EvolveGCN-H: dùng GRU để cập nhật weight của GCNConv qua các timestep.
        
        Args:
            in_channels:  số chiều input feature
            hidden_channels: số chiều embedding hidden
            num_layers: số lớp GCNConv (thường 2)
        """
        super(EvolveGCNH, self).__init__()
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers

        # Lớp GCN ban đầu
        self.gcn_layers = nn.ModuleList()
        self.gcn_layers.append(GCNConv(in_channels, hidden_channels))
        for _ in range(1, num_layers):
            self.gcn_layers.append(GCNConv(hidden_channels, hidden_channels))

        # GRU cập nhật weight động
        self.gru = nn.GRUCell(hidden_channels, hidden_channels)

        # Linear để phân loại (node classification)
        self.classifier = nn.Linear(hidden_channels, 2)

        # Trạng thái ẩn ban đầu của GRU
        self.register_buffer("h", torch.zeros(hidden_channels))

    def forward(self, x, edge_index):
        """
        Forward 1 snapshot (graph tại timestep t).
        """
        h_out = x
        for gcn in self.gcn_layers:
            h_out = gcn(h_out, edge_index)
            h_out = F.relu(h_out)

        # Cập nhật trọng số động qua GRU
        self.h = self.gru(h_out.mean(dim=0), self.h)

        # Áp classifier cho node classification
        out = self.classifier(h_out)
        return out

In [28]:
from torch.optim.lr_scheduler import StepLR
device = torch.device("xpu" if torch.xpu.is_available() else "cpu")
# device = torch.device("cpu")
in_channels = train_snapshots[0].x.size(1)   # số feature
hidden_channels = 64

model = EvolveGCNH(in_channels, hidden_channels).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)  # mỗi 20 epoch giảm lr còn 0.5 * lr
criterion = nn.CrossEntropyLoss()

# Training loop qua nhiều timestep
for epoch in range(1, 501):
    model.train()
    total_loss = 0
    for snapshot in train_snapshots:   # train_snapshots: list các Data object PyG cho từng timestep
        x, edge_index, y = snapshot.x.to(device), snapshot.edge_index.to(device), snapshot.y.to(device)
        optimizer.zero_grad()
        out = model(x, edge_index)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss={total_loss:.4f}")
    scheduler.step()


Epoch 10, Loss=4.4815
Epoch 20, Loss=3.7752
Epoch 30, Loss=2.7085
Epoch 40, Loss=2.5980
Epoch 50, Loss=2.1712
Epoch 60, Loss=2.1999
Epoch 70, Loss=1.6293
Epoch 80, Loss=1.5891
Epoch 90, Loss=1.3555
Epoch 100, Loss=1.2833
Epoch 110, Loss=1.2001
Epoch 120, Loss=1.1687
Epoch 130, Loss=1.1220
Epoch 140, Loss=1.1008
Epoch 150, Loss=1.0824
Epoch 160, Loss=1.0711
Epoch 170, Loss=1.0614
Epoch 180, Loss=1.0555
Epoch 190, Loss=1.0506
Epoch 200, Loss=1.0475
Epoch 210, Loss=1.0449
Epoch 220, Loss=1.0434
Epoch 230, Loss=1.0420
Epoch 240, Loss=1.0412
Epoch 250, Loss=1.0405
Epoch 260, Loss=1.0401
Epoch 270, Loss=1.0398
Epoch 280, Loss=1.0396
Epoch 290, Loss=1.0394
Epoch 300, Loss=1.0393
Epoch 310, Loss=1.0392
Epoch 320, Loss=1.0391
Epoch 330, Loss=1.0391
Epoch 340, Loss=1.0391
Epoch 350, Loss=1.0390
Epoch 360, Loss=1.0390
Epoch 370, Loss=1.0390
Epoch 380, Loss=1.0390
Epoch 390, Loss=1.0390
Epoch 400, Loss=1.0390
Epoch 410, Loss=1.0390
Epoch 420, Loss=1.0390
Epoch 430, Loss=1.0390
Epoch 440, Loss=1.03

KeyboardInterrupt: 

In [ ]:
import numpy as np
import torch
from sklearn.metrics import precision_recall_fscore_support, f1_score, confusion_matrix

@torch.no_grad()
def reset_hidden_state(model, device):
    """Reset HIDDEN STATE (không reset trọng số đã học). 
    Tự động xử lý một số biến phổ biến của các bản EvolveGCN custom."""
    # EvolveGCNH bản tự code có thể có current_W (vector hoá của W) và/hoặc h
    if hasattr(model, "h"):
        # buffer dạng [hidden_dim] hoặc [*, hidden_dim]
        model.h = torch.zeros_like(model.h, device=device)
    if hasattr(model, "current_W"):
        # nếu có W0 ban đầu thì đưa current_W về W0; 
        # nếu bạn muốn giữ trạng thái đã train, KHÔNG gọi reset_hidden_state trước eval.
        if hasattr(model, "W0"):
            model.current_W = model.W0.view(-1).to(device)
        else:
            model.current_W = torch.zeros_like(model.current_W, device=device)

def _forward_one(model, snapshot, device, classifier=None):
    """Forward 1 snapshot, trả về logits (không tính loss)."""
    x = snapshot.x.to(device)
    edge_index = snapshot.edge_index.to(device)
    out = model(x, edge_index)           # embedding hoặc logits tùy kiến trúc
    if classifier is not None:
        logits = classifier(out)         # nếu out là embedding
    else:
        logits = out                     # nếu model trả logits trực tiếp
    return logits, snapshot.y.cpu().numpy()

@torch.no_grad()
def evaluate_sequence(model, snapshots, device, name="TEST", classifier=None, warm_start=True, per_step=False):
    """
    Eval tuần tự qua các snapshot theo thời gian:
      - KHÔNG dùng nhãn để cập nhật tham số (no backward)
      - Vẫn cập nhật HIDDEN STATE qua forward (RNN-style)
    Args:
      model: EvolveGCN model (custom hoặc PyG-Temporal)
      snapshots: list[Data] theo thứ tự thời gian
      device: torch.device
      name: nhãn in log
      classifier: nn.Module hoặc None (nếu model trả logits trực tiếp)
      warm_start: True -> giữ nguyên hidden state sau training; False -> reset hidden trước eval
      per_step: True -> in metric từng timestep
    """
    model.eval()
    if classifier is not None:
        classifier.eval()

    if not warm_start:
        reset_hidden_state(model, device)

    all_preds, all_labels = [], []
    if per_step:
        print(f"=== {name} per-timestep ===")

    for t_idx, snap in enumerate(snapshots, 1):
        logits, y_true_np = _forward_one(model, snap, device, classifier)
        preds_np = logits.argmax(dim=1).cpu().numpy()

        all_preds.append(preds_np)
        all_labels.append(y_true_np)

        if per_step:
            prec_t, rec_t, f1_t, _ = precision_recall_fscore_support(y_true_np, preds_np, zero_division=0, labels=[0,1])
            micro_t = f1_score(y_true_np, preds_np, average="micro", zero_division=0)
            print(f"t={t_idx:02d}: Prec0={prec_t[0]:.3f} Rec0={rec_t[0]:.3f} F1_0={f1_t[0]:.3f}  Micro={micro_t:.3f}")

    y_true_all = np.concatenate(all_labels)
    y_pred_all = np.concatenate(all_preds)

    prec, rec, f1, _ = precision_recall_fscore_support(y_true_all, y_pred_all, zero_division=0, labels=[0,1])
    micro_f1 = f1_score(y_true_all, y_pred_all, average="micro", zero_division=0)
    cm = confusion_matrix(y_true_all, y_pred_all, labels=[0,1])

    print(f"\n=== {name} (aggregated) ===")
    print("Class 1 - Precision: %.3f  Recall: %.3f  F1: %.3f" % (prec[0], rec[0], f1[0]))
    print("Micro-F1: %.3f" % micro_f1)
    print("Confusion Matrix [rows=true 0/1]:\n", cm)

    return {
        "precision_per_class": prec,
        "recall_per_class": rec,
        "f1_per_class": f1,
        "micro_f1": micro_f1,
        "confusion_matrix": cm,
    }


In [ ]:
evaluate_sequence(model, test_snapshots, device, name="TEST", classifier=None, warm_start=True, per_step=True)
